# Uso de RDD's avanzado
En Big Data es habitual trabajar con datos en formato clave-valor. Por ello, Spark ofrece transformaciones y acciones específicas para estos casos

In [1]:
# Inicializamos SparkSession y SparkContext
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("spark://spark-master:7077") \
    .appName("Jupyter-Spark") \
    .config("spark.eventLog.enabled", "true") \
    .config("spark.eventLog.dir", "file:///opt/spark/logs/history") \
    .config("spark.history.fs.logDirectory", "file:///opt/spark/logs/history") \
    .getOrCreate()

spark.version  # Verifica la versión de Spark

#spark = SparkSession.builder.getOrCreate()

sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/05 17:05:14 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [9]:
rdd_pares1 = sc.parallelize([('a', 1), ('b', 1), ('c', 1)])
print (rdd_pares1.collect())

rdd_st = sc.parallelize ("Big Data aplicado. Curso de especialización de Inteligencia Artificial y Big Data".split())
rdd_pares2 = rdd_st.map(lambda palabra: (palabra,1))
print (rdd_pares2.collect())

[('a', 1), ('b', 1), ('c', 1)]
[('Big', 1), ('Data', 1), ('aplicado.', 1), ('Curso', 1), ('de', 1), ('especialización', 1), ('de', 1), ('Inteligencia', 1), ('Artificial', 1), ('y', 1), ('Big', 1), ('Data', 1)]


Lo interesante de los RDD's de pares clave valor es que proporcionar una serie de transformaciones y acciones adicionales
## Transformaciones
### keyBy
Función que crea una clave para cada valor actual de un RDD

In [25]:
# La clave es la inicial de cada palabra
rdd_pares = rdd_st.keyBy(lambda palabra: palabra[0])
rdd_pares.collect()
rdd_pares3 = rdd_st.keyBy (lambda palabra: len(palabra))
print (rdd_pares3.collect())

[(3, 'Big'), (4, 'Data'), (9, 'aplicado.'), (5, 'Curso'), (2, 'de'), (15, 'especialización'), (2, 'de'), (12, 'Inteligencia'), (10, 'Artificial'), (1, 'y'), (3, 'Big'), (4, 'Data')]


### mapValues
Realiza una operación map sólo sobre los valores del RDD

In [12]:
rdd_pares.mapValues(lambda x: x.upper()).collect()

[('B', 'BIG'),
 ('D', 'DATA'),
 ('a', 'APLICADO.'),
 ('C', 'CURSO'),
 ('d', 'DE'),
 ('e', 'ESPECIALIZACIÓN'),
 ('d', 'DE'),
 ('I', 'INTELIGENCIA'),
 ('A', 'ARTIFICIAL'),
 ('y', 'Y'),
 ('B', 'BIG'),
 ('D', 'DATA')]

### groupByKey
Agrupa los valores en función de la clave

In [16]:
rdd_pares.groupByKey().collect()

[('e', <pyspark.resultiterable.ResultIterable at 0x7f16e00c36d0>),
 ('B', <pyspark.resultiterable.ResultIterable at 0x7f16d160bb10>),
 ('I', <pyspark.resultiterable.ResultIterable at 0x7f16d1609d50>),
 ('A', <pyspark.resultiterable.ResultIterable at 0x7f16d1609cd0>),
 ('D', <pyspark.resultiterable.ResultIterable at 0x7f16e2df39d0>),
 ('d', <pyspark.resultiterable.ResultIterable at 0x7f16c031bf10>),
 ('y', <pyspark.resultiterable.ResultIterable at 0x7f16d1e33110>),
 ('a', <pyspark.resultiterable.ResultIterable at 0x7f16d160a6d0>),
 ('C', <pyspark.resultiterable.ResultIterable at 0x7f16d1609c50>)]

### reduceByKey
Aplica una función reductora después de agrupar los valores del RDD en función de la clave

In [21]:
rdd_pares2.reduceByKey(lambda x,y: x+y).collect()

[('Curso', 1),
 ('aplicado.', 1),
 ('especialización', 1),
 ('Big', 2),
 ('de', 2),
 ('y', 1),
 ('Inteligencia', 1),
 ('Artificial', 1),
 ('Data', 2)]

### sortByKey
Ordena

In [29]:
rdd_pares3.sortByKey(ascending=False).collect()

[(15, 'especialización'),
 (12, 'Inteligencia'),
 (10, 'Artificial'),
 (9, 'aplicado.'),
 (5, 'Curso'),
 (4, 'Data'),
 (4, 'Data'),
 (3, 'Big'),
 (3, 'Big'),
 (2, 'de'),
 (2, 'de'),
 (1, 'y')]

### join
Hay varios tipos posibles de join
- Inner join: join()
- Full outer join: fullOuterJoin()
- left outer join: leftOuterJoin()
- right outer join: rightOuterJoin()
- producto cartesiano: cartesian() (no se recomienda su uso)

In [2]:
rdd1 = sc.parallelize([('a', 1), ('b', 2), ('c', 3)])
rdd2 = sc.parallelize([('a', 4), ('b', 5), ('c', 6)])
rdd3 = rdd1.join(rdd2)
print(rdd3.collect())

rdd4 = rdd1.leftOuterJoin(rdd2)
print (rdd2.collect())

[('c', (3, 6)), ('a', (1, 4)), ('b', (2, 5))]
[('a', 4), ('b', 5), ('c', 6)]


### union
Permite unir varios RDD's

In [8]:
rdda = sc.parallelize ([('a',1),('b',2),('b',3)])
rddb = sc.parallelize ([('a',3),('b',1),('c',2)])
rdd_union = rdda.union(rddb)
rdd_union.collect()

[('a', 1), ('b', 2), ('b', 3), ('a', 3), ('b', 1), ('c', 2)]

## Acciones
### lookup
Devuelve sólo los valores que coinciden con la clave especificada

In [13]:
rdd_pares.lookup('B')

['Big', 'Big']

### countByKey
Permite contar el número de valores que se corresponden con una determinada clave

In [18]:
rdd_pares.countByKey()

defaultdict(int,
            {'B': 2,
             'D': 2,
             'a': 1,
             'C': 1,
             'd': 2,
             'e': 1,
             'I': 1,
             'A': 1,
             'y': 1})